## Regression Trees

In [1]:
import pandas as pd 

import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [67]:
df = pd.read_csv("final.csv", sep=";")

In [70]:
df.head(10)

,FID_x,IID,log_BMI,SEX_y,AGE,index_OfBMI,Age_group,Age_index,famid,pid,...,SNP325.1,SNP325.2,SNP326.1,SNP326.2,SNP327.1,SNP327.2,SNP328.1,SNP328.2,SNP329.1,SNP329.2
0,3,1,1.410102,2,64,2,60 - 69,6,3,1,...,G,G,T,T,G,G,G,G,G,G
1,4,2,1.391112,2,29,1,20 - 29,2,4,2,...,A,G,T,T,G,G,G,G,G,G
2,5,3,1.381837,1,64,1,60 - 69,6,5,3,...,G,G,T,T,G,G,G,G,G,G
3,6,4,1.290702,2,21,1,20 - 29,2,6,4,...,G,G,T,T,G,G,G,G,G,G
4,7,5,1.494572,2,38,2,30 - 39,3,7,5,...,G,G,T,T,G,G,G,G,G,G
5,9,7,1.417804,1,43,2,40 - 49,4,9,7,...,G,G,T,T,A,G,A,G,A,G
6,10,8,1.343014,2,51,1,50 - 59,5,10,8,...,G,G,T,T,G,G,G,G,G,G
7,11,9,1.273001,2,20,1,20 - 29,2,11,9,...,A,G,T,T,G,G,G,G,A,G
8,12,10,1.559068,1,45,2,40 - 49,4,12,10,...,G,G,T,T,G,G,0,0,G,G
9,13,11,1.359646,2,27,1,20 - 29,2,13,11,...,G,G,T,T,G,G,G,G,G,G


In [71]:
colList = []
for i in df.columns:
    if i.startswith('SNP'):
        colList.append(i)

In [72]:
for i in colList:
    df[i] = df[i].factorize()[0]

In [74]:
df

,FID_x,IID,log_BMI,SEX_y,AGE,index_OfBMI,Age_group,Age_index,famid,pid,...,SNP325.1,SNP325.2,SNP326.1,SNP326.2,SNP327.1,SNP327.2,SNP328.1,SNP328.2,SNP329.1,SNP329.2
0,3,1,1.410102,2,64,2,60 - 69,6,3,1,...,0,0,0,0,0,0,0,0,0,0
1,4,2,1.391112,2,29,1,20 - 29,2,4,2,...,1,0,0,0,0,0,0,0,0,0
2,5,3,1.381837,1,64,1,60 - 69,6,5,3,...,0,0,0,0,0,0,0,0,0,0
3,6,4,1.290702,2,21,1,20 - 29,2,6,4,...,0,0,0,0,0,0,0,0,0,0
4,7,5,1.494572,2,38,2,30 - 39,3,7,5,...,0,0,0,0,0,0,0,0,0,0
5,9,7,1.417804,1,43,2,40 - 49,4,9,7,...,0,0,0,0,1,0,1,0,1,0
6,10,8,1.343014,2,51,1,50 - 59,5,10,8,...,0,0,0,0,0,0,0,0,0,0
7,11,9,1.273001,2,20,1,20 - 29,2,11,9,...,1,0,0,0,0,0,0,0,1,0
8,12,10,1.559068,1,45,2,40 - 49,4,12,10,...,0,0,0,0,0,0,2,1,0,0
9,13,11,1.359646,2,27,1,20 - 29,2,13,11,...,0,0,0,0,0,0,0,0,0,0


pandas.core.frame.DataFrame

In [64]:
df

(array([0, 0, 0, ..., 0, 0, 0]), Int64Index([0, 1, 2], dtype='int64'))

In [53]:
df["SEX"] = df["SEX_y"]-1
feats = colList + ["SEX_y", "Age_index"]

0       1
1       1
2       0
3       1
4       1
5       0
6       1
7       1
8       0
9       1
10      1
11      1
12      0
13      1
14      0
15      1
16      1
17      1
18      1
19      0
20      1
21      0
22      0
23      1
24      0
25      0
26      1
27      1
28      1
29      1
       ..
5513    1
5514    0
5515    0
5516    0
5517    0
5518    0
5519    1
5520    1
5521    1
5522    1
5523    0
5524    0
5525    0
5526    0
5527    1
5528    1
5529    0
5530    0
5531    0
5532    1
5533    0
5534    0
5535    1
5536    1
5537    1
5538    0
5539    1
5540    0
5541    1
5542    0
Name: SEX, Length: 5543, dtype: int64

In [36]:
X = df[feats].values
y = df["log_BMI"].values

In [49]:
X

array([[0, 0, 0, ..., 0, 2, 6],
       [0, 0, 0, ..., 0, 2, 2],
       [0, 0, 1, ..., 0, 1, 6],
       ...,
       [0, 0, 1, ..., 0, 1, 3],
       [1, 0, 0, ..., 0, 2, 3],
       [0, 0, 0, ..., 0, 1, 2]])

In [45]:
model = DecisionTreeRegressor(max_depth=10)
scores = cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error")


(-0.0647518229025456, 0.0014944382702189137)

In [48]:
print("ACC %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() *2))

ACC -0.08 (+/- 0.00)
